# Modelo ARIMA **'Precio Media Anual'** Propiedades

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "queretaro" in nombre
    and ('financiamientos' not in nombre and 'poblacion' not in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
        # Renombrar la columna
    df.rename(columns={'Recamaras':'recamaras','Estacionamiento':'estacionamiento','m_total': 'metros_total','m_construido':'metros_construido','tiempo_publicacion':'tiempo_de_publicacion','Estacionamientos':'estacionamiento','Recamaras': 'recamaras', 
       'baño':'Banos','medio_baño':'Medio_banos', 'Seguridad':'seguridad','seguridad_privada':'seguridad','baño_total':'Banos_Total', 'cp':'CP',                  'Colonia':'colonia','Estado':'estado','Tipo':'tipo','Categoria':'categoria','Precio':'precio','m2_contruido': 'm2_construido','segmento':'categoria','Categoria':'categoria','Category':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria', 'precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)

In [4]:
import re

# Función para extraer mes y año del nombre del DataFrame
def extraer_mes_y_ano(nombre):
    # Expresión regular para capturar tanto meses completos como abreviados
    match = re.search(r'(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)_(\d{4})', nombre)
    if match:
        mes = match.group(1).lower()  # Mes en texto
        ano = int(match.group(2))  # Año en formato numérico
        return mes, ano
    return None, None

# Diccionario de traducción de meses de texto a números
meses_dict = {
    'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12,
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

df_resultados = pd.DataFrame(columns=['categoria', 'precio', 'mes', 'año'])
for nombre_df in nombres_df_filtrados:
    df = globals()[nombre_df]
    
    # Extraer mes y año del nombre del DataFrame
    mes_str, ano = extraer_mes_y_ano(nombre_df)
    
    # Verificar que se haya extraído el mes y el año correctamente
    if mes_str is not None and ano is not None:
        mes = meses_dict.get(mes_str)  # Convertir el nombre del mes en su número correspondiente
        
        # Añadir las columnas de mes y año al DataFrame actual
        df['mes'] = mes
        df['año'] = ano
        df_resultados = pd.concat([df_resultados, df], ignore_index=True)
df_resultados[['categoria','precio','mes','año']]


,categoria,precio,mes,año
0,B1,2640000,4,2024
1,S2,6950000,4,2024
2,B2,2908782,4,2024
3,E1,280000,4,2024
4,E1,400000,4,2024
...,...,...,...,...
46467,C1,2230000,6,2023
46468,D1,1380000,6,2023
46469,C3,1710000,6,2023
46470,C2,1843000,6,2023


In [8]:
columnas_object = df_resultados.select_dtypes(include='object').columns
columnas_object

Index(['categoria', 'precio', 'mes', 'año', 'propiedad', 'estado', 'tipo',
       'seguridad', 'colonia', 'CP', 'url', 'q', 'status', 'fraccionamiento'],
      dtype='object')

#### PRECIO

In [5]:
df_resultados['precio'] = pd.to_numeric(df_resultados['precio'], errors='coerce')
df_resultados['precio'] = df_resultados['precio'].fillna(0)
print(df_resultados['precio'].dtype)

int64


#### CATEGORÍA

In [6]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

df_resultados = df_resultados.drop(columns=['categoria'])
df_resultados['categoria'] = df_resultados['precio'].apply(asignar_categoria)
df_resultados['categoria'] = df_resultados['categoria'].fillna('')

#print(df_resultados['categoria'].unique())
# Crear la columna 'id_categoria' convirtiendo 'categoria' en valores numéricos
df_resultados['id_categoria'] = pd.factorize(df_resultados['categoria'])[0]
# Mostrar nueva columna 'id_categoria'
#print(df_resultados[['categoria', 'id_categoria']])
# Imprimir el diccionario
print("Diccionario de categorías: ")
diccionario_categorias = dict(zip(df_resultados['id_categoria'], df_resultados['categoria']))
for id_categoria, categoria in diccionario_categorias.items():
    print(f"{id_categoria}: {categoria}")

Diccionario de categorías: 
0: B1
1: S2
2: B2
3: E1
4: E2
5: 
6: E3
7: D1
8: D2
9: ELITE
10: D3
11: C1
12: C2
13: C3
14: A3
15: A2
16: A1
17: B3
18: S1
19: L3
20: S3
21: L1
22: L+
23: L2


#### MESES

In [7]:
df_resultados['mes'] = pd.to_numeric(df_resultados['mes'], errors='coerce')
df_resultados['mes'] = df_resultados['mes'].fillna(0)

print(df_resultados['mes'].dtype)

int64


#### AÑO

In [8]:
df_resultados['año'] = pd.to_numeric(df_resultados['año'], errors='coerce')
df_resultados['año'] = df_resultados['año'].fillna(0)
print(df_resultados['año'].dtype)

int64


#### ESTADO

In [9]:
df_resultados['estado'].str.lower().unique()

array(['venta', nan], dtype=object)

In [10]:
# Convertir 'Estado' en 1 si es 'VENTA' y 0 si es cualquier otro valor, incluido NaN
df_resultados['venta'] = df_resultados['estado'].apply(lambda x: 1 if x == 'venta' else 0)
df_resultados['venta'].unique()

array([0], dtype=int64)

#### TIPO

In [11]:
df_resultados['tipo'] = df_resultados['tipo'].fillna('')
categorias = [df_resultados['tipo'].unique()]
categorias

[array(['Casa', 'Departamento', 'Casa En Fraccionamiento', 'Duplex',
        'Condominio Horizontal', 'Casa En Condominio', 'Casa Duplex'],
       dtype=object)]

In [12]:
mapeo_categorias = {
    'Casa': 'casa',
    'Casa En Fraccionamiento': 'casa',
    'Casa En Condominio': 'casa',
    'Departamento': 'departamento',
    'Condominio Horizontal': 'departamento',
    'casa': 'casa',
    'Casa ': 'casa'
}
df_resultados['tipo'] = df_resultados['tipo'].replace(mapeo_categorias)

In [13]:
# Imprimir el diccionario
df_resultados['id_tipo'] = pd.factorize(df_resultados['tipo'])[0]
print("Diccionario de tipos: ")
diccionario_categorias = dict(zip(df_resultados['id_tipo'], df_resultados['tipo']))
for id_categoria, categoria in diccionario_categorias.items():
    print(f"{id_categoria}: {categoria}")

Diccionario de tipos: 
0: casa
1: departamento
2: Duplex
3: Casa Duplex


#### CP

In [14]:
df_resultados['CP'].unique()

array(['', 0, 76047, 76245, 76246, 76269, 76178, 76240, 76147, 76267,
       76249, 76150, 76060, 76220, 76902, 76243, 76134, 76910, 76903,
       76000, 76904, 76900, 76906, 76907, 76908, 76117, 76125, 76185,
       76922, 76086, 76915, 76146, 76085, 76803, 76905, 76148, 76529,
       76116, 76190, 76226, 79610, 76010, 76177, 76923, 76400, 76230,
       76264, 76244, 76912, 79246, 76920, 76179, 76950, 76812, 76258,
       94486, 76421, 76218, 76237, 76930, 76135, 76180, 76924, 76474],
      dtype=object)

In [15]:
df_resultados['CP'] = df_resultados['CP'].fillna('')
mapeo = {
    0.0: 0,
    '' : 0,
}
df_resultados['CP'] = df_resultados['CP'].replace(mapeo).infer_objects(copy=False)
print(df_resultados['CP'].dtype)

int64


C:\Users\yoe11\AppData\Local\Temp\ipykernel_9108\1467107239.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resultados['CP'] = df_resultados['CP'].replace(mapeo).infer_objects(copy=False)


____
## 

In [16]:
df_resultados['fecha'] = pd.to_datetime(df_resultados['año'].astype(str) + '-' + df_resultados['mes'].astype(str) + '-01')
df_resultados[['fecha','mes','año']]

,fecha,mes,año
0,2024-04-01,4,2024
1,2024-04-01,4,2024
2,2024-04-01,4,2024
3,2024-04-01,4,2024
4,2024-04-01,4,2024
...,...,...,...
46467,2023-06-01,6,2023
46468,2023-06-01,6,2023
46469,2023-06-01,6,2023
46470,2023-06-01,6,2023


In [17]:
precio_media_anual_pachuca = df_resultados.groupby('año')['precio'].median().reset_index()
precio_media_anual_pachuca.rename(columns={'precio': 'promedio'}, inplace=True)
precio_media_anual_pachuca

,año,promedio
0,2023,3150000.0
1,2024,4400000.0


In [18]:
# Utilizar pct_change para calcular el cambio porcentual entre 2023 y 2024
diferencia_porcentual = precio_media_anual_pachuca['promedio'].pct_change().iloc[1] * 100

# Proyecciones a los siguientes años, Comenzando con 2024
predicciones = pd.DataFrame()
predicciones[2024] = [precio_media_anual_pachuca[precio_media_anual_pachuca['año'] == 2024]['promedio'].iloc[0]]

# Proyectar los siguientes años aplicando la diferencia porcentual acumulativa
for año in range(2025, 2028):
    # Aplicar el crecimiento porcentual sobre el valor del año anterior
    predicciones[año] = round(predicciones[año-1] * (1 + diferencia_porcentual / 100), 2)
df_predicciones = pd.DataFrame(predicciones)
df_predicciones = df_predicciones.reset_index()
predict_pachuca = pd.melt(df_predicciones, id_vars='index', var_name='año', value_name='promedio')
predict_pachuca.drop('index', axis=1, inplace=True)
predict_pachuca

,año,promedio
0,2024,4400000.00
1,2025,6146031.75
2,2026,8584933.24
3,2027,11991652.78


In [19]:
df_concatenado = pd.concat([precio_media_anual_pachuca, predict_pachuca], ignore_index=True)
df_concatenado['promedio'] = df_concatenado['promedio'].round()
df_concatenado

,año,promedio
0,2023,3150000.0
1,2024,4400000.0
2,2024,4400000.0
3,2025,6146032.0
4,2026,8584933.0
5,2027,11991653.0


In [25]:
# Datos
año = df_concatenado['año']
promedio = df_concatenado['promedio']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[2000000, 14000000],  # Establecer el rango del eje y
        tickvals=[2000000, 4000000, 6000000, 8000000, 10000000, 12000000, 14000000],  # Definir los valores de las marcas en el eje y
        ticktext=['$2,000,000.00','$4,000,000.00', '$6,000,000.00','$8,000,000.00','$10,000,000.00','$12,000,000.00','$14,000,000.00',],  # Definir el texto de las marcas en el eje y
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()

## MODELO DE REGRESIÓN LINEAL

In [41]:
columnas_num = df_resultados.select_dtypes(include=['int64', 'float64','datetime']).columns
columnas_num

Index(['precio', 'mes', 'año', 'id', 'metros_total', 'metros_construido',
       'precio_m2_terreno', 'precio_m2_construido', 'tiempo_de_publicacion',
       'estacionamiento', 'recamaras', 'Banos', 'Medio_banos', 'Banos_Total',
       'seguridad', 'CP', 'ids', 'personas_interesadas',
       'promedio_interes_mensual', 'cantidad_pisos', 'antiguedad',
       'id_categoria', 'venta', 'id_tipo', 'fecha'],
      dtype='object')

#### PREPARAR VARIABLES

Dividir los datos en conjuntos de entrenamiento y prueba

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# Definir las variables independientes (X) y la variable dependiente (y)
X = df_resultados[['mes','año','metros_total','metros_construido','precio_m2_terreno','precio_m2_construido','tiempo_de_publicacion','estacionamiento','recamaras','Banos','Medio_banos','Banos_Total','CP','seguridad','id_categoria', 'venta', 'id_tipo']]
y = df_resultados['precio']
# Conjuntos de entrenamiento y prueba
# X_train y X_test: Conjuntos de entrenamiento y prueba para las variables independientes
# y_train y y_test: Conjuntos de entrenamiento y prueba para la variable dependiente
# test_size=0.2: 20% de los datos serán utilizados para el conjunto de prueba                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# random_state=42: Semilla para la aleatoriedad, asegura reproducibilidad de los resultados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Crear y entrenar el modelo

In [48]:
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

#### Crear predicciones, Calcular promedio de precios predichos 2027

In [49]:
# Hacer predicciones
predicciones = modelo.predict(X_test)

# Calcular el promedio de precios predichos por mes y año
df_predicciones = pd.DataFrame({'año': X_test['año'], 'mes': X_test['mes'], 'precio_predicho': predicciones})
promedios_predichos = df_predicciones.groupby(['año', 'mes'])['precio_predicho'].mean().reset_index()
print(promedios_predichos)

     año  mes  precio_predicho
0   2022    5     2.133041e+06
1   2022    7     1.412685e+06
2   2022   10     1.750357e+06
3   2023    3     1.548780e+06
4   2023    5     1.407224e+06
5   2023    6     2.891987e+06
6   2023    7     1.377911e+06
7   2023    9     2.347023e+06
8   2023   10     2.205943e+06
9   2024    2     1.732104e+06
10  2024    3     1.263594e+06
11  2024    4     1.072034e+06
12  2024    5     1.179029e+06
13  2024    6     1.393208e+06
14  2024    7     1.414143e+06
15  2024    8     1.109247e+06


# BACKND